# Модуль 4. Стрктуры данных
## Абстрактные структуры данных: Очередь

## Практикум

### Ваше решение

In [1]:
from abc import ABC, abstractmethod
from typing import List
import random


class Queue:

    def __init__(self):
        self.__data = list()

    def enqueue(self, value):
        if type(value) is list:
            self.__data.extend(value)
        else:
            self.__data.append(value)

    def dequeue(self):
        if len(self.__data) > 0:
            return self.__data.pop(0)

        return None

    def rear(self):
        if len(self.__data) > 0:
            return self.__data[-1]
        return None

    def front(self):
        if len(self.__data) > 0:
            return self.__data[0]
        return None

    def is_empty(self):
        return len(self.__data) == 0

    def size(self, ):
        return len(self.__data)

    def clear(self, ):
        self.__data = list()

    def __repr__(self):
        arry = [str(val) for val in self.__data]
        return ' '.join(arry)


class FighterAbstract(ABC):

    def __init__(self, name: str):
        self.name = name
        self.health = 100
        self.is_alive = True
        self.weight = round(random.uniform(100.0, 200.0))
        self.growth = round(random.uniform(50.0, 200.0))
        self.power = int(self.weight * 2 / self.growth)

    @abstractmethod
    def attack(self, fighter: 'FighterAbstract') -> bool:
        '''Проводит атаку на соперника'''
        pass

    @abstractmethod
    def show_info(self) -> None:
        '''Печатает информацию о бойце в произвольном формате'''
        pass


class Figter(FighterAbstract):

    @staticmethod
    def block():

        chance = [True, False]
        block_chance = random.choice(chance)
        return block_chance

    def attack(self, fighter: 'FighterAbstract') -> bool:

        if self.block():
            print(f'{fighter.name}:   Удар заблокировал')
            return False

        else:
            demange = random.randrange(1, 20)
            fighter.health -= demange
            print(f'{fighter.name}:   Отхватил: {demange}, Здоровья: {fighter.health}')
            return True

    def show_info(self) -> None:
        print(f'{self.name}:   Рост: {self.weight}, Вес: {self.growth}, Сила: {self.power}')

    def __repr__(self):
        return f'{self.name}'


class ContestAbstract(ABC):
    winner: FighterAbstract

    @classmethod
    @abstractmethod
    def new(cls, first: FighterAbstract, second: FighterAbstract) -> None:
        '''Инициализация боя'''

    @classmethod
    @abstractmethod
    def fight(cls) -> None:
        '''Здесь описывается логика боя'''

    def __new__(_):
        raise Exception('Невозможно создать экземпляр класса!')


class Contest(ContestAbstract):
    fighters = []

    @staticmethod
    def print_first_move(fighter: FighterAbstract) -> None:
        print(f'{fighter.name} атакует первым')
        print('------------------------------------')

    @classmethod
    def new(cls, first: FighterAbstract, second: FighterAbstract) -> None:
        yokozunas = [first, second]

        yokozunas[0].show_info()
        yokozunas[1].show_info()
        print('------------------------------------')

        if yokozunas[0].power < yokozunas[1].power:
            cls.fighters.append(yokozunas.pop(0))
            cls.print_first_move(cls.fighters[0])
            cls.fighters.append(yokozunas.pop(0))

        elif yokozunas[0].power > yokozunas[1].power:
            cls.fighters.append(yokozunas.pop(1))
            cls.print_first_move(cls.fighters[0])
            cls.fighters.append(yokozunas.pop(0))

        else:
            cls.fighters.append(yokozunas.pop(random.randrange(0, 1)))
            cls.print_first_move(cls.fighters[0])
            cls.fighters.append(yokozunas.pop(0))

    @classmethod
    def fight(cls) -> None:
        while True:
            if cls.fighters[1].health > 0:
                if cls.fighters[0].attack(cls.fighters[1]):
                    continue

                else:
                    cls.fighters.reverse()

            else:
                cls.fighters[1].is_alive = False
                cls.winner = cls.fighters[0]
                break

    def __new__(cls, *args):
        cls.new(*args)


# Contest.new(Figter('Петя'), Figter('Вася'))
# Contest.fight()
# print("Победил боец", Contest.winner.name)


class TeamsContestAbstract(Contest, ABC):
    @classmethod
    @abstractmethod
    def new(cls, file_of_fighters: str, countries: List[str]):
        pass

    @classmethod
    @abstractmethod
    def draw(cls):
        '''Жеребьёвка участников'''


class TeamsContest(TeamsContestAbstract):
    all_winers = []
    fighters = []
    draw_list = []
    team1 = {}
    team2 = {}

    @classmethod
    def new(cls, file_of_fighters: str, countries: List[str]) -> None:

        cls.team1[countries[0]] = list()
        cls.team2[countries[1]] = list()

        with open(file_of_fighters, 'r') as file:
            for name in file.readlines():
                fighter = name.split(' ')
                fighter_name = f'{fighter[1]} {fighter[2]}'.replace('\n', '')

                if fighter[0] == countries[0]:
                    value = cls.team1.get(countries[0])
                    value.append(fighter_name)
                    cls.team1.update({countries[0]: value})

                elif fighter[0] == countries[1]:
                    value = cls.team2.get(countries[1])
                    value.append(fighter_name)
                    cls.team2.update({countries[1]: value})


    @classmethod
    def draw(cls) -> None:
        countries = list(cls.team1.keys()) + list(cls.team2.keys())

        # перемешивание
        one = cls.team1[countries[0]]
        one = random.sample(one, len(one))
        two = cls.team2[countries[1]]
        two = random.sample(two, len(two))

        for fighter1, fighter2 in zip(one, two):
            cls.draw_list.append((fighter1, fighter2))

            value1 = cls.team1.get(countries[0])
            value1.remove(fighter1)
            cls.team1.update({countries[0]: value1})

            value2 = cls.team2.get(countries[1])
            value2.remove(fighter2)
            cls.team2.update({countries[1]: value2})

        print('Образовались следующие пары:')
        print('------------------------------------')

        for i in cls.draw_list:
            print(f'{i[0]} и {i[1]}')

        print('====================================')

    @classmethod
    def start(cls, first: FighterAbstract, second: FighterAbstract) -> None:
        yokozunas = [first, second]

        yokozunas[0].show_info()
        yokozunas[1].show_info()
        print('------------------------------------')

        if yokozunas[0].power < yokozunas[1].power:
            cls.fighters.append(yokozunas.pop(0))
            cls.print_first_move(cls.fighters[0])
            cls.fighters.append(yokozunas.pop(0))

        elif yokozunas[0].power > yokozunas[1].power:
            cls.fighters.append(yokozunas.pop(1))
            cls.print_first_move(cls.fighters[0])
            cls.fighters.append(yokozunas.pop(0))

        else:
            cls.fighters.append(yokozunas.pop(random.randrange(0, 1)))
            cls.print_first_move(cls.fighters[0])
            cls.fighters.append(yokozunas.pop(0))


    @classmethod
    def fight(cls) -> None:
        for set in cls.draw_list:
            print('На бой вызываются:')

            cls.start(Figter(set[0]), Figter(set[1]))

            while True:
                if cls.fighters[1].health > 0:
                    if cls.fighters[0].attack(cls.fighters[1]):
                        continue

                    else:
                        cls.fighters.reverse()

                else:
                    cls.fighters[1].is_alive = False
                    cls.winner = cls.fighters[0]
                    break

            print(f'Победил боец {cls.winner.name}')
            cls.all_winers.append(cls.winner.name)
            cls.fighters.clear()
            print('====================================')

        countries = list(cls.team1.keys()) + list(cls.team2.keys())
        one = cls.team1.get(countries[0])
        two = cls.team2.get(countries[1])

        if len(one) > 0:
            print(f'В команде {countries[0]} в поединке не участвовали {len(one)}')
            for name in one:
                print(name)

        elif len(two) > 0:
            print(f'В команде {countries[1]} в поединке не участвовали {len(two)}')
            for name in two:
                print(name)

        else:
            print('Все бойцы участвовали в поединке!')
        print('====================================')
        print('Поединок закончен')
        print('=============Победители=============')
        for name in cls.all_winers:
            print(name)


TeamsContest.new("fighters.txt", ["RU", "UK"])
TeamsContest.draw()

Образовались следующие пары:
------------------------------------
Стас Михайлов и Мик Джаггер
Борис Гребенщиков и Paul McCartney
Андрей Макаревич и Ozzy Osbourne
Борис Моисеев и Elton John


### Самопроверка

In [2]:
TeamsContest.fight()

На бой вызываются:
Стас Михайлов:   Рост: 175, Вес: 158, Сила: 2
Мик Джаггер:   Рост: 135, Вес: 200, Сила: 1
------------------------------------
Мик Джаггер атакует первым
------------------------------------
Стас Михайлов:   Отхватил: 5, Здоровья: 95
Стас Михайлов:   Отхватил: 11, Здоровья: 84
Стас Михайлов:   Удар заблокировал
Мик Джаггер:   Удар заблокировал
Стас Михайлов:   Отхватил: 12, Здоровья: 72
Стас Михайлов:   Отхватил: 9, Здоровья: 63
Стас Михайлов:   Удар заблокировал
Мик Джаггер:   Удар заблокировал
Стас Михайлов:   Отхватил: 9, Здоровья: 54
Стас Михайлов:   Удар заблокировал
Мик Джаггер:   Отхватил: 11, Здоровья: 89
Мик Джаггер:   Отхватил: 11, Здоровья: 78
Мик Джаггер:   Отхватил: 19, Здоровья: 59
Мик Джаггер:   Отхватил: 11, Здоровья: 48
Мик Джаггер:   Удар заблокировал
Стас Михайлов:   Отхватил: 1, Здоровья: 53
Стас Михайлов:   Отхватил: 19, Здоровья: 34
Стас Михайлов:   Отхватил: 16, Здоровья: 18
Стас Михайлов:   Отхватил: 13, Здоровья: 5
Стас Михайлов:   Отхватил: 